# Extracting Data from TMDB

   - There is no financial information included in the IMDB data (e.g. budget or revenue).
   - This will be a major roadblock when attempting to analyze which movies are successful and must be addressed before you will be able to determine which movies are successful.
   - My stakeholder identified The Movie Database (TMDB) as a great source of financial data (https://www.themoviedb.org/). Thankfully, TMDB offers a free API for programmatic access to their data!
   - The stakeholder has asked me to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

# Specifications - Financial Data

   - My stakeholder has asked me to extract and save the results for movies that meet all of the criteria established in part 1 of the project.
   - As a proof-of-concept, they requested I perform a test extraction of movies that started in 2000 or 2001.
   - Each year should be saved as a separate .csv.gz file.

In [1]:
#Imports
import numpy as np
import pandas as pd
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

# My API Credentials

In [2]:
with open('/Users/Nick/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
#Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [3]:
tmdb.API_KEY =  login['api-key']

# API Calls

In [4]:
#Movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

In [5]:
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/ncEsesgOJDNrTUED89hYbA117wo.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 78.472,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

In [6]:
info['budget']

63000000

In [7]:
info['revenue']

463517383

In [8]:
info['imdb_id']

'tt0133093'

In [9]:
#Budget of Tom and Jerry
movie = tmdb.Movies('tt1361336')
info = movie.info()
info['budget']

50000000

In [10]:
#example from package README - rating if in the US
response = movie.releases()
for c in movie.countries:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])

PG
PG
PG


# Prep for API Calls

In [11]:
#Designating Folder
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'akas.csv.gz',
 'basics.csv.gz',
 'ratings.csv.gz',
 'tmdb_api_results_2000.json']

In [12]:
#Define the years
YEARS_TO_GET = list(range(2000,2022))

# Custom Functions

In [13]:
# Function to obtain movie rating
def get_movie_with_rating(movie_id):
    # Get movie and release dates
    movie = tmdb.Movies(movie_id)
    # Construct output dictionary
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            movie_info['certification'] = c['certification']
    return movie_info

In [14]:
#Function to append to JSON File
#Adapted from:https://www.geeksforgeeks.org/append-to-json-file-using-python/
def write_json(new_data, filename):    
    with open(filename,'r+') as file:
        #First we load existing data into a dict.
        file_data = json.load(file)
        #Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        #Sets file's current position at offset.
        file.seek(0)
        #convert back to json.
        json.dump(file_data, file)

# Extracting Data

In [15]:
#OUTER LOOP
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    # Load in the dataframe from project part 1 as basics:
    basics = pd.read_csv('Data/basics.csv.gz')
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

#INNER LOOP
    # Get index and movie id from list
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue

YEARS:   0%|          | 0/22 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1333 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1517 [00:00<?, ?it/s]

Movies from 2002:   0%|          | 0/1505 [00:00<?, ?it/s]

Movies from 2003:   0%|          | 0/1630 [00:00<?, ?it/s]

...

Movies from 2017:   0%|          | 0/5510 [00:00<?, ?it/s]

Movies from 2018:   0%|          | 0/5629 [00:00<?, ?it/s]

Movies from 2019:   0%|          | 0/5675 [00:00<?, ?it/s]

Movies from 2020:   0%|          | 0/4764 [00:00<?, ?it/s]

Movies from 2021:   0%|          | 0/4701 [00:00<?, ?it/s]

In [16]:
for YEAR in YEARS_TO_GET:
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

In [17]:
final_year_df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0199602,0.0,/fOnjxCbR3Poar7voHcaQrvYqQMH.jpg,None,0.0,"[{'id': 35, 'name': 'Comedy'}]",,894346.0,en,Holy Hollywood,...,0.0,82.0,[],Released,,Holy Hollywood,0.0,0.0,0.0,
2,tt0293429,0.0,/9yBVqNruk6Ykrwc32qrK2TIE5xw.jpg,"{'id': 931431, 'name': 'Mortal Kombat Collecti...",20000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 14, 'nam...",https://www.mortalkombatmovie.net,460465.0,en,Mortal Kombat,...,83601013.0,110.0,"[{'english_name': 'Japanese', 'iso_639_1': 'ja...",Released,Get over here.,Mortal Kombat,0.0,7.2,4614.0,R
3,tt0326716,0.0,/iXpy7O02lPPR97meAMKOSQBmebD.jpg,None,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",http://5-25-77themovie.com,199734.0,en,5-25-77,...,0.0,135.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some days are just better than others.,5-25-77,0.0,5.0,5.0,NR
4,tt0499097,0.0,/fPGeS6jgdLovQAKunNHX8l0avCy.jpg,None,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",https://www.amazon.com/dp/B08VFD1Y3B,567189.0,en,Tom Clancy's Without Remorse,...,0.0,109.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,From the author of Rainbow Six.,Tom Clancy's Without Remorse,0.0,7.1,1911.0,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,tt15234108,0.0,/814oAzF3CH1tPSsehVnQZgiIDOD.jpg,None,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 12, 'name...",https://www.chorki.com/bn/videos/networkerbaire/,860039.0,bn,নেটওয়ার্কের বাইরে,...,0.0,89.0,"[{'english_name': 'Bengali', 'iso_639_1': 'bn'...",Released,,Networker Baire,0.0,7.5,4.0,NaN
1279,tt15237900,0.0,/uzYERLhms6R9aXkQUsekCrB0u7t.jpg,None,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",https://www.maverickentertainment.cc/movies/a-...,912765.0,en,A Very Merry New Year,...,0.0,156.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,To pour decisions and broken resolutions.,A Very Merry New Year,0.0,0.0,0.0,
1280,tt15237914,0.0,/br3VPHg3SI3uJZKiy85kbiVv4J0.jpg,None,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",,855096.0,ru,Солнцепёк,...,0.0,133.0,"[{'english_name': 'Russian', 'iso_639_1': 'ru'...",Released,,Hotsunlight,0.0,7.4,14.0,NaN
1281,tt15241706,0.0,None,None,0.0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",,861165.0,en,Noah’s Shark,...,0.0,0.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Noah’s Shark,0.0,4.0,1.0,
